#Actionable Insights from a Data Lake, Satisfy GDPR

we will try to merge lot of historical data plus incoming streaming data.

As per regulatory agency has decreed you be able to identify and delete all data associated with a specific user (i.e. GDPR).

In other words, you must delete data associated with a specific deviceId.

In [2]:
%run "./Reference/Setup"

In [3]:
inputPath = "/mnt/training/gaming_data/mobile_streaming_events_b"
outputPathBronze = userhome + "/gaming/bronze01"
outputPathSilver = userhome + "/gaming/silver"
outputPathGold = userhome + "/gaming/gold"

### Data Preparation and Read Streaming Data

The input source is a folder containing files of around 100,000 bytes each and hence  made it slow stream (just for demo).

In [5]:
from pyspark.sql.types import StructType, StringType, IntegerType, TimestampType, DoubleType

eventSchema = ( StructType()
  .add('eventName', StringType()) 
  .add('eventParams', StructType() 
    .add('game_keyword', StringType()) 
    .add('app_name', StringType()) 
    .add('scoreAdjustment', IntegerType()) 
    .add('platform', StringType()) 
    .add('app_version', StringType()) 
    .add('device_id', StringType()) 
    .add('client_event_time', TimestampType()) 
    .add('amount', DoubleType()) 
  )     
)

gamingEventDF = (spark
  .readStream
  .schema(eventSchema) 
  .option('streamName','mobilestreaming_demo') 
  .option("maxFilesPerTrigger", 1)                # treat each file as Trigger event
  .json(inputPath) 
)

### Write Stream

* Write the stream from `gamingEventDF` to the Databricks Delta data lake in path defined by `outputPathBronze`.
* Convert `client_event_time` to a date format and rename to `eventDate`
* Filter out null `eventDate`

In [7]:
from pyspark.sql.functions import to_date, col

eventsStream = (gamingEventDF
  .withColumn('eventDate', to_date(gamingEventDF.eventParams.client_event_time))
  .filter(col('eventDate').isNotNull())
  .writeStream 
  .partitionBy('eventDate') 
  .format('delta') 
  .option('checkpointLocation', outputPathBronze + '/_checkpoint') 
  .start(outputPathBronze)
)

In [8]:
# wait for stream to initialize
spark.sql("""
    DROP TABLE IF EXISTS mobile_events_delta_raw
  """)
spark.sql("""
    CREATE TABLE mobile_events_delta_raw
    USING DELTA 
    LOCATION '{}' 
  """.format(outputPathBronze))

Out[ 32 ]: DataFrame[]

### Create a Databricks Delta table

Creating device_id_type_table from data in /mnt/training/gaming_data/dimensionData.

This table associates deviceId with deviceType = {android, ios}.

In [10]:

tablePath = "/mnt/training/gaming_data/dimensionData"
spark.sql("""
    DROP TABLE IF EXISTS device_id_type_table
  """)
spark.sql("""
    CREATE TABLE device_id_type_table
    USING DELTA 
    LOCATION '{}' 
  """.format(tablePath))

Out[ 33 ]: DataFrame[]

### Create a query table

Create table `mobile_events_delta_query` by joining `device_id_type_table` with `mobile_events_delta_raw` on `deviceId`.
* Fields are `eventName`, `deviceId`, `eventTime`, `eventDate` and `deviceType`.
* recommed to use `PARTITION BY (eventDate)`
* Write to `outputPathSilver`

In [12]:
# ANSWER
spark.sql("""
    DROP TABLE IF EXISTS mobile_events_delta_query
""")

spark.sql("""
    CREATE TABLE mobile_events_delta_query
    USING DELTA
    PARTITIONED BY (eventDate)
    LOCATION '{}'
    AS 
      SELECT eventName, eventParams.device_id AS deviceId, eventParams.client_event_time AS eventTime, eventDate, device_id_type_table.deviceType AS deviceType
      FROM mobile_events_delta_raw
      INNER JOIN device_id_type_table ON device_id_type_table.device_id = mobile_events_delta_raw.eventParams.device_id
""".format(outputPathSilver))             

Out[ 34 ]: DataFrame[]

### Create a Delta summary table out of query table

The company executives want to look at the number of active users by week.

Count number of events in the by week.

In [14]:
spark.sql("""
    DROP TABLE IF EXISTS mobile_events_delta_summary
""")

spark.sql("""
    CREATE TABLE mobile_events_delta_summary 
    USING DELTA
    LOCATION '{}'
    AS
      SELECT count(DISTINCT deviceId) AS WAU, weekofyear(eventTime) as week
      FROM mobile_events_delta_query
      GROUP BY weekofyear(eventTime)
      ORDER BY weekofyear(eventTime)
""".format(outputPathGold))   

Out[ 35 ]: DataFrame[]

### Visualization

The company executives are visual people: they like pretty charts like everyone :).

Let's add bar chart out of `mobile_events_delta_summary` where the horizontal axis s month and the vertical axis is WAU.

In [16]:
%sql

SELECT * FROM mobile_events_delta_summary

WAU,week
4,4
9,38
11,28
16,21
7,45
13,27
3,47
15,24
4,7
5,11


### remove specific `deviceId`

Identify all the events associated with a specific user, rougly proxied by the first `deviceId` we encounter in our query. 

Use the `mobile_events_delta_query` table.

The `deviceId` you come up with should be a string.

In [18]:
deviceId = str(spark.sql("SELECT deviceId FROM mobile_events_delta_query limit 1").collect()[0][0])

-sandbox
### Using ZORDER 

Since the events are implicitly ordered by `eventTime`, implicitly re-order by `deviceId`. 

The data pertaining to this `deviceId` is spread out all over the data lake. (It's definitely _not_ co-located!).

Pass in the `deviceId` variable you defined above.

`ZORDER` may take a few minutes.

In [20]:
%sql

OPTIMIZE mobile_events_delta_query
ZORDER BY (deviceId)

path
null


### Time to Delete Specific `deviceId`

0. Delete rows with that particular `deviceId` from `mobile_events_delta_query`.
0. Make sure that `deviceId` is no longer in the table!

In [22]:
# ANSWER
spark.sql("DELETE FROM mobile_events_delta_query WHERE deviceId='{}' ".format(deviceId))
noDeviceId = spark.sql(("SELECT * FROM mobile_events_delta_query WHERE deviceId='{}' ").format(deviceId)).collect()
print(noDeviceId)

[]

-sandbox
### Clean Up
 
We use 0 hours here for purposes of demonstration only.

Recall, we use `VACUUM` to reduce the number of files in each partition directory to 1.

In [24]:
for streamingQuery in spark.streams.active:
  streamingQuery.stop()

In [25]:
%sql
VACUUM mobile_events_delta_query RETAIN 0 HOURS;

path
dbfs:/user/nagaraj.sengodan@hotmail.com/gaming/silver
